In [1]:
import numpy as np
import pandas as pd

###  ID3 ( Entropy)

In [ ]:
#     a1 = data[hedef].value_counts()[0]/len(data[hedef])
#     a2 = data[hedef].value_counts()[1]/len(data[hedef])

In [2]:
data = pd.read_csv("covid_naive_bayes.csv")
# data.drop("Unnamed: 0",axis=1,inplace=True)
data.head()

,Cinsiyet,Yaş,Sigara,Maske,Sonuc
0,Kadın,29,Hayır,Yok,negatif
1,Erkek,71,Hayır,Yok,pozitif
2,Erkek,39,Hayır,Yok,negatif
3,Kadın,24,Evet,Yok,pozitif
4,Erkek,32,Evet,Yok,negatif


In [95]:
data["Sonuc"].value_counts()[0]/len(data.Sonuc)

0.6

In [76]:
data["Cinsiyet"].unique()[0]

'Kadın'

In [86]:
for i in range(len(data["Cinsiyet"].unique())):
    pivot = data.groupby(['Sonuc']).apply(lambda x: x[x['Cinsiyet'] == data["Cinsiyet"].unique()[i] ].count())
    pi = pivot.Sonuc.pozitif
    ni = pivot.Sonuc.negatif
    print(pivot)

         Cinsiyet  Yaş  Sigara  Maske  Sonuc
Sonuc                                       
negatif         5    5       5      5      5
pozitif         5    5       5      5      5
         Cinsiyet  Yaş  Sigara  Maske  Sonuc
Sonuc                                       
negatif        13   13      13     13     13
pozitif         7    7       7      7      7


In [3]:
def Info_D(pi,ni):
    if pi*ni == 0:
        result =0
    else:
        a1,a2 = pi/(pi+ni),ni/(pi+ni)
        result=a1*np.log2(a1)+a2*np.log2(a2)
    return round(-1*result,3)
def Info_Attribute_D(data,target,column):
    total = 0
    for i in range(len(data[column].unique())):
        pivot = data.groupby([target]).apply(lambda y: y[y[column] == data[column].unique()[i] ].count())
        pi = pivot[column][0]
        ni = pivot[column][1]
        total += ((pi+ni)/len(data[target]))*Info_D(pi,ni)
    data[target]
    # print(f"Info_{column}_(D) = ",round(total,3))
    s1,s2 = data[target].value_counts()[0],data[target].value_counts()[1]
    print(f"Gain({column}): ",round((Info_D(s1,s2)-total),3))
Info_Attribute_D(data,"Sonuc","Sigara")

Gain(Sigara):  0.02


In [103]:
Info_Attribute_D(d1,"buys_computer","credit_rating")
Info_Attribute_D(d1,"buys_computer","age")
Info_Attribute_D(d1,"buys_computer","income")
Info_Attribute_D(d1,"buys_computer","student")

Gain(credit_rating):  0.048
Gain(age):  0.246
Gain(income):  0.029
Gain(student):  0.151


In [185]:
Info_Attribute_D(data,"Sonuc","Sigara")

Info_Sigara_(D) =  0.9512


In [186]:
Info_Attribute_D(data,"Sonuc","Maske")

Info_Maske_(D) =  0.9661333333333333


In [197]:
# d1 = pd.DataFrame((np.zeros(70).reshape(14,5)),columns=["age","income","student","credit_rating","buys_computer"])
# l = ["<=30","<=30","31..40",">40",">40",">40","31..40","<=30","<=30",">40","<=30","31..40","31..40",">40"]
# l = ["high","high","high","medium","low","low","low","medium","low","medium","medium","medium","high","medium"]
# l = "no no no no yes yes yes no yes yes yes no yes no"
# l = l.split(" ")
# l = "fair excellent fair fair fair excellent excellent fair fair fair excellent excellent fair excellent".split()
# l = "no no yes yes yes no yes no yes yes yes yes yes no".split()
# d1["age"] = l

In [22]:
d1 = pd.read_csv("buys_computer.csv")

In [12]:
d1["age"].unique()

array(['<=30', '31..40', '>40'], dtype=object)

In [71]:
round(0.940-0.911,3)

0.029

###  CART  (GİNİ)

In [2]:
sk = pd.read_csv("play_golf.csv")
sk.head(3)

,Outlook,Temp,Humidity,Wind,Decision
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes


In [74]:
def gini(data,target,column):
    total = 0
    if len(data[target].unique()) <2:
        return total
    else:
        for i in range(len(data[column].unique())):
            pivot = data.groupby([target]).apply(lambda y: y[y[column] == data[column].unique()[i] ].count())
            pi,ni = pivot[column][0],pivot[column][1]
            g = 1-pow(pi/(pi+ni),2)-pow(ni/(pi+ni),2)
            total += (pi+ni)/len(data[target])*g
    return round(total,3)

def main(data,target,column):
    ginis = {}
    tree = {}
    tree[column] = list(data[column].unique())
    new_data = data.drop(target,axis=1)
    new_cols = list(new_data.columns)
    child = tree[column]
    new_cols.remove(column)
    for k in child:
        data1 = data[data[f"{column}"]== f"{k}"]
        for i in new_cols:
            result = gini(data1,target,i)
            ginis[i] = result
            if result == 0 and type(result) == int:
                tree[k] = data1[target].unique()[0]
            else:
                build_tree(data,ginis,k,tree)
        if tree[k] == data[target].unique()[0] or tree[k] == data[target].unique()[1]:
            pass
        else:
            for z in data1[tree[k]].unique():
                result2= gini(data1[data1[tree[k]]== f"{z}"],target,tree[k])
                if result2 == 0 and type(result2) == int:
                    tree[z] = data1[data1[tree[k]]== f"{z}"][target].unique()[0]
                else:
                    build_tree(data,ginis,z,tree)
        print("ginis:",ginis)
    return tree
def build_tree(data,ginis,k,tree):
    key = sorted(ginis.items(),key= lambda x: x[1])[0][0]
    value = list(data[key].unique())
    tree[k] = key
    
def root(data,target):
    ginis = {}
    new_data = data.drop(target,axis=1)
    for i in new_data.columns:
        ginis[i] = gini(data,i,target)
    column = sorted(ginis.items(),key= lambda x: x[1])[0][0]
    return column

<p style="color: red; font-size:17px;">root fonskiyonu datayı ve target alarak kök düğümü bulur. gini fonksiyonu cart algoritmasında kullanılan gini metriğini hesaplar ve yuvarlayarka return eder. main fonksiyonu ise root fonksiyonundan kök düğümü alır ve yavruları ile ağaca ekler. Daha sonra yavrular üzerinde gezinerek bulunan düğüm hariç diğer düğümlerin gini değerleri hesaplanır  ve bu gini değerleri tree ile  build_tree fonksiyonuna gönderilir. build_tree bu gini değelerini sıralar ve en küçük gini değerini ağaca ekler.
</p>

In [49]:
tel = pd.read_csv("buys_computer.csv")

In [75]:
main(tel,"buys_computer",root(tel,"buys_computer"))

ginis: {'income': 0.2, 'student': 0.0, 'credit_rating': 0.467}
ginis: {'income': 0, 'student': 0, 'credit_rating': 0}
ginis: {'income': 0.467, 'student': 0.467, 'credit_rating': 0.0}


{'age': ['<=30', '31..40', '>40'],
 '<=30': 'student',
 'no': 'no',
 'yes': 'yes',
 '31..40': 'yes',
 '>40': 'credit_rating',
 'fair': 'yes',
 'excellent': 'no'}

In [76]:
main(sk,"Decision",root(sk,"Decision"))

ginis: {'Temp': 0.2, 'Humidity': 0.0, 'Wind': 0.467}
ginis: {'Temp': 0, 'Humidity': 0, 'Wind': 0}
ginis: {'Temp': 0.467, 'Humidity': 0.467, 'Wind': 0.0}


{'Outlook': ['Sunny', 'Overcast', 'Rain'],
 'Sunny': 'Humidity',
 'High': 'No',
 'Normal': 'Yes',
 'Overcast': 'Yes',
 'Rain': 'Wind',
 'Weak': 'Yes',
 'Strong': 'No'}

In [72]:
main(sk,"Decision",root(sk,"Decision"))

ginis: {'Temp': 0.2, 'Humidity': 0.0, 'Wind': 0.467}
ginis: {'Temp': 0, 'Humidity': 0, 'Wind': 0}
ginis: {'Temp': 0.467, 'Humidity': 0.467, 'Wind': 0.0}


{'Outlook': ['Sunny', 'Overcast', 'Rain'],
 'Sunny': 'Humidity',
 'High': 'No',
 'Normal': 'Yes',
 'Overcast': 'Yes',
 'Rain': 'Wind',
 'Weak': 'Yes',
 'Strong': 'No'}

In [298]:
kale = sk[sk["Outlook"]=="Sunny"]
# kale[kale["Outlook"] == "Sunny"]
kale[kale["Humidity"] == "High"]

,Outlook,Temp,Humidity,Wind,Decision
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
7,Sunny,Mild,High,Weak,No


In [36]:
denem = []
denem.append(["deneme","ali"])
denem.append(["kale"])
print(denem)
denem[0].remove("deneme")
print(denem)
denem[0].remove("ali")
print(denem)
print(len(denem))

[['deneme', 'ali'], ['kale']]
[['ali'], ['kale']]
[[], ['kale']]
2


In [59]:
# def gini(data,target,column):
#     total = 0
#     if len(data[target].unique()) <2:
#         return total
#     else:
#         for i in range(len(data[column].unique())):
#             pivot = data.groupby([target]).apply(lambda y: y[y[column] == data[column].unique()[i] ].count())
#             pi,ni = pivot[column][0],pivot[column][1]
#             if pi*ni == 0:
#                 pass
#             g = 1-pow(pi/(pi+ni),2)-pow(ni/(pi+ni),2)
#             total += (pi+ni)/len(data[target])*g
#     return round(total,3)

# def main(data,target,column):
#     ginis = {}
#     q = []
#     tree = {}
#     tree[column] = list(data[column].unique())
#     q.append(list(tree.keys()))
#     new_data = data.drop(target,axis=1)
#     new_cols = list(new_data.columns)
#     for c in q[0]:
#         child = tree[c]
#         q.append(child)
#         new_cols.remove(c)
#         for k in child:
#             data1 = data[data[f"{c}"]== f"{k}"]
#             for i in new_cols:
#                 result = gini(data1,target,i)
#                 ginis[i] = result
#                 if result == 0 and type(result) == int:
#                     tree[k] = data1[target].unique()[0]
#                 else:
#                     build_tree(data,ginis,k,tree)
#             if tree[k] == data[target].unique()[0] or tree[k] == data[target].unique()[1]:
#                 pass
#             else:
#                 for z in data1[tree[k]].unique():
#                     result2= gini(data1[data1[tree[k]]== f"{z}"],target,tree[k])
#                     if result2 == 0 and type(result2) == int:
#                         tree[z] = data1[data1[tree[k]]== f"{z}"][target].unique()[0]
#                     else:
#                         build_tree(data,ginis,z,tree)
#             print("ginis:",ginis)
#         print("q: ",q)
#         q[0].remove(c)
#         print("q: ",q)
#     return tree
# def build_tree(data,ginis,k,tree):
#     key = sorted(ginis.items(),key= lambda x: x[1])[0][0]
#     value = list(data[key].unique())
#     tree[k] = key
    
# def root(data,target):
#     ginis = {}
#     new_data = data.drop(target,axis=1)
#     for i in new_data.columns:
#         ginis[i] = gini(data,i,target)
#     column = sorted(ginis.items(),key= lambda x: x[1])[0][0]
#     return column